Extracting User's Google Calendar

In [21]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery


In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import methods #file containing functions to interact with Google calendar like add event, create event, etc
import json
import re

google_calendar = methods.get_google_calendar_service() #get the google calendar service
calendar = methods.extractCalendar(google_calendar) #extract the calendar from the service
client = methods.set_up_ChatGPT(calendar) #set up the ChatGPT model
now = methods.current_time()

Here are the events for today:
1. SI 649 from 4:00 PM to 5:30 PM
2. Jogging Session from 2:00 AM to 3:00 AM
3. Jogging Session from 2:00 AM to 3:00 AM
4. Jogging Session from 4:00 AM to 5:00 AM

Get User's Query

In [24]:
context = f"Here's my calendar for the week, store it for your reference. Today's date and time is: {now}\n{calendar}"
instruction = """\nAs my virtual assistant, you are an expert with managing the calendar.
You can create, move, check, list, update, delete, and handle recurring events based on the user's requests.
When the user doesn't provide enough information, you can ask for more details, upto 3 times. Otherwise use your best judgement. 
When the user wants to create an event or move an event and if the user hasn't specified when they want it scheduled, then refer to the calendar and use your best judgement to schedule the event. Keep in mind when the user would normally eat or sleep. Assume default duration for any event as 1 hour.
Check for any conflicts before scheduling an event.
Always ask for confirmation before making any changes to the calendar, if the user doesn't agree, if the user doesn't agree, then generate another suitable time slot.
When you are done, please let the user know that the task has been completed, and don't ask any more questions. If you aren't done, end with a question mark.
Please perform the necessary actions using the Google Calendar API.\n"""

# initial setup
initial_prompt = context + instruction

In [25]:
user_request = input()
combined_prompt = initial_prompt + "\nUser request: " + user_request


In [26]:
def chat_gpt(client, messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content

messages = [
    {
        "role": "system",
        "content": initial_prompt
    },
    { 
        "role": "user",
        "content": user_request
    }
]

In [27]:
finalized_response = False

#establishing feedback loop to chat with user 
while not finalized_response:
    response = chat_gpt(client, messages)
    print(response)
    messages.append({
        "role": "assistant",
        "content": response
    })
    if re.compile(r'\?').search(response):
        finalized_response = False
        user_feedback = input()
        messages.append({
            "role": "user",
            "content": user_feedback
        })
    else:
        finalized_response = True


Sure, I see that you have some free time tomorrow. Would you like me to schedule a jogging session for you tomorrow?
Great! I will schedule a jogging session for you tomorrow. Is there a specific time you prefer for the jog, or shall I suggest a time based on your usual schedule?
Based on your usual schedule, I see that you have some free time in the morning. I will schedule the jogging session for tomorrow at 8:00 AM. Does that work for you?
Got it! I'll schedule the jogging session for tomorrow at 9:00 AM. Does that time work for you?
Great! Your jogging session has been scheduled for tomorrow at 9:00 AM. Task completed.


In [31]:
#get GPT to output the function calls I need to make to the calendar API
instruction2 = """\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text. 
All dates should be in 'YYYY-MM-DD' format, and times should be in 24-hour format 'HH:MM'. Additionally, for functions requiring a datetime range (such as 'checkSchedule' and 'ListEvents'), the datetime should be in ISO 8601 format 'YYYY-MM-DDTHH:MM:SSZ'.
Available function templates to include in the response string:
- createEvent(description, date:default=today, time, duration:default=1)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_description, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, date, time, duration, frequency, until)
- deleteRecurringEvent(event_id)
- updateRecurringEvent(event_id, new_description, new_duration, new_location)

Make sure to fill in the required information, such as event names, IDs, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.

Example input: 'Schedule a team meeting for Friday afternoon at 3pm'
Example output: 'createEvent("Team Meeting", "YYYY-MM-DD", "15:00", 1)'

Now, please generate the necessary function call(s) based on the user's request."""

In [32]:
messages.append({
    "role": "system",
    "content": instruction2
})
function_call = chat_gpt(client, messages)
print(function_call)
#Example of function call: createEvent("Meeting with John", "2022-12-31", "10:00 AM", 2)

createEvent("Jogging Session", "2024-03-27", "09:00", 1)


In [33]:
function_calls = [line.strip() for line in function_call.split(';')] 
print(function_calls)

# Define a dictionary mapping function names to their corresponding Python functions
functions = {
    "createEvent": methods.createEvent,
    "moveEvent": methods.moveEvent,
    "checkSchedule": methods.checkSchedule,
}

#TODO: works for createEvent, gotta change other functions in methods file to accept arguments as strings and convert them to appropriate data types
# Iterate through the function calls
for call in function_calls:
    # Split the string to extract function name and arguments
    parts = call.split('(')
    function_name = parts[0]
    arguments = parts[1].rstrip(')').split(', ')
    
    # Call the corresponding function with the provided arguments
    if function_name in functions:
        functions[function_name](*arguments)
    else:
        print(f"Function '{function_name}' not found.")

['createEvent("Jogging Session", "2024-03-27", "09:00", 1)']
Event created: Jogging Session on 2024-03-27 at 09:00 for duration of 1 hours.
